# The [Raku-ju](https://raku.land/cpan:BDUGGAN/Jupyter::Kernel) hijack hack for [D3.js](https://d3js.org)


## aka *[JavaScript::D3](https://raku.land/zef:antononcube/JavaScript::D3) demo*


Anton Antonov  
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)  
December 2022

## Load packages
Here we load some packages that are used to generate, summarize, and modify datasets:

In [1]:
use Data::Generators;
use Data::Reshapers;
use Data::Summarizers;
use Data::ExampleDatasets;

This loads the "JavaScript::D3" package:

In [2]:
use JavaScript::D3;

## Setup
Here we use a JavaScript cell that allows the visualization of with D3.js in Jupyter notebooks:

In [3]:
%% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

-------

## Random data examples

In [4]:
%%js
js-d3-list-plot( (^120)>>.rand )

In [5]:
%%js
js-d3-list-line-plot( random-real(30, 1000) ):!grid-lines

In [6]:
%%js
js-d3-list-plot(
    transpose([random-real((100, 120),100), random-variate( NormalDistribution.new(12,19), 100)]), 
    x-axis-label=>'random-real', 
    y-axis-label=>'N[12,19]',
    title=> 'Real 2D data')

-----

## Iris dataset

In [7]:
my @dsIris = example-dataset( /iris $/);
records-summary(@dsIris);
@dsIris.&dimensions

+------------------+--------------------+----------------+--------------------+--------------------+-----------------+
| Species          | Sepal.Length       | rownames       | Petal.Width        | Sepal.Width        | Petal.Length    |
+------------------+--------------------+----------------+--------------------+--------------------+-----------------+
| setosa     => 50 | Min    => 4.3      | Min    => 1    | Min    => 0.1      | Min    => 2        | Min    => 1     |
| versicolor => 50 | 1st-Qu => 5.1      | 1st-Qu => 38   | 1st-Qu => 0.3      | 1st-Qu => 2.8      | 1st-Qu => 1.6   |
| virginica  => 50 | Mean   => 5.843333 | Mean   => 75.5 | Mean   => 1.199333 | Mean   => 3.057333 | Mean   => 3.758 |
|                  | Median => 5.8      | Median => 75.5 | Median => 1.3      | Median => 3        | Median => 4.35  |
|                  | 3rd-Qu => 6.4      | 3rd-Qu => 113  | 3rd-Qu => 1.8      | 3rd-Qu => 3.3      | 3rd-Qu => 5.1   |
|                  | Max    => 7.9      | Max   

(150 6)

In [8]:
%%js
js-d3-histogram(@dsIris.map(*<Petal.Length>), title=>'Petal.Length')

In [9]:
%%js
js-d3-list-plot(@dsIris.map({ ($_.<Petal.Length>, $_.<Petal.Width>) }), title=>'Petal')

In [10]:
%%js
js-d3-list-plot(@dsIris.map({ %(x=>$_.<Petal.Length>, y=>$_.<Petal.Width>, group => $_<Species>) }), title=>'Petal'):legends

-----

## Air quality dataset

In [11]:
my @dsAir = example-dataset('airquality');
records-summary(@dsAir);
@dsAir.&dimensions

+---------------------+---------------------+--------------------+---------------------+--------------------+-----------------+----------------------+
| Temp                | Day                 | Month              | Ozone               | Wind               | rownames        | Solar.R              |
+---------------------+---------------------+--------------------+---------------------+--------------------+-----------------+----------------------+
| Min    => 56        | Min    => 1         | Min    => 5        | Min    => 0         | Min    => 1.7      | Min    => 1     | Min    => 0          |
| 1st-Qu => 72        | 1st-Qu => 8         | 1st-Qu => 6        | 1st-Qu => 2.5       | 1st-Qu => 7.4      | 1st-Qu => 38.5  | 1st-Qu => 93.5       |
| Mean   => 77.882353 | Mean   => 15.803922 | Mean   => 6.993464 | Mean   => 31.941176 | Mean   => 9.957516 | Mean   => 77    | Mean   => 177.424837 |
| Median => 79        | Median => 16        | Median => 7        | Median => 21        | Media

(153 7)

In [12]:
%%js
js-d3-bubble-chart(select-columns(@dsAir, <Temp Wind Ozone>)>>.values>>.List, grid-lines=>12)

In [13]:
%%js
js-d3-bubble-chart(
rename-columns(select-columns(@dsAir, <Wind Temp Ozone Month>), (<Wind Temp Ozone Month> Z=> <x y z group>).Hash))

------

## The execution loop

```mermaid
graph TD
   Raku{{Raku}}
   IRaku{{"Raku<br>Jupyter kernel"}}
   Jupyter{{Jupyter}}
   JS{{JavaScript}}
   RakuInput[/Raku code input/]
   JSOutput[/JavaScript code output/]
   CellEval[Cell evaluation]
   JSResDisplay[JavaScript code result display]
   Jupyter -.-> |1|IRaku -.-> |2|Raku -.-> |3|JSOutput -.-> |4|Jupyter
   Jupyter -.-> |5|JS -.-> |6|JSResDisplay
   RakuInput ---> CellEval ---> Jupyter  ---> JSResDisplay
```